### Na dzisiejszych zajęcia będziemy szukać potencjałów wywołanych, dokladniej N400. 

What I normally follow as pre-processing steps are:
1. Reference - your online and offline reference may be different, and different options can be used in the process. Some are referring the signal to electrodes on mastoids, some to Cz, some to the average of all the electrodes.
2. Filters - you probably already used some filters during data collection. Often, online filters are less restrict than what you may want to apply in the offline pre-processing. Consider something like 0.1 Hz high-pass, and 40 Hz low-pass filters.
3. Segmentation (or epoching, depending on software) - "cutting" the signal to pieces depending on your time-windows of interest and triggers (or markers, again - a choice of terminology). For example, if your trial consists of a visual stimulus presented for 1000ms and followed by 500ms ITI, you may want to "cut out" segments of -100ms to 1000ms from the onset of the stimulus (marked by a trigger).
4. Baseline correction - the "-100ms" from the previous point will serve now as baseline for the segments. Of course, you can consider a different time window for your baseline. 100 ms is quite standard, though.
5. Blink correction - this can be done in many ways. ICA (Independent Component Analysis) is often used for this purpose. There are plenty of sources online about how it works, and many softwares have implemented algorithms for using ICA on your data.
6. Artifact rejection - blink correction is already a part of artifact rejection, but there's more to this category. You may want to go through your signal and remove all the other artifacts (like movements, floating electrodes, sweat, etc).
7. Averaging - you may want to average the signal for each participant across conditions. This reduces noise. Or you may want to get into single trial EEG (again, many sources online).
8. Choosing ROIs - if you're planning to choose a Region of Interest, you can decide on a few electrodes that would serve as such. Again, there are different ways of how to make these decisions. Often people look at the topographies of their signal and compare that to the previous literature. Deciding on your ROI, you can average the signal from these electrodes and you end up with a datapoint per condition per participant.

Źródło: https://www.researchgate.net/post/How_to_evaluate_N170_N200_P300_N400_and_P600_from_EEG_signals

### Zacznijmy od importu potrzebnych paczek. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import aseegg as ag 
import numpy as np
import random

### Plik, który będziemy używać to 'dane_potencjaly.csv'

In [ ]:
df = pd.read_csv('dane_potencjaly.csv', names=['FCz', 'Cz', 'Fz', 'Pz', 'C3', 'C4', 'F3', 'F4','event'])

In [ ]:
df

<img src="../images/elektrody_eeg.png" width="500" align="center">

(Kaongoen & Jo, 2017)

<img src="../images/TRex-RAS.png" width="600" align="center">

### Opis eventów
1 - successful click - udane wciśnięcie spacji, skutkujące skokiem w grze (gra trwała dalej) <br>
2 - unsuccessful click - nieudane wciśnięcie spacji, program nie wykonał polecenia skoku (gra trwała dalej)<br>
3 - successful click before endgame - udane wciśnięcie spacji w programie, które prowadziło do przegranej gry (błąd użytkownika)<br>
4 - unsuccessful click before endgame - nieudane wciśnięcie spacji, które prowadziło do przegranej gry (błąd interfejsu)<br>
5 - succesful click endgame - moment kolizji w grze, który był poprzedzony udanym wciśnięciem spacji (feedback o błędzie)<br>
6 - unsuccesful click endgame - moment kolizji w grze, który był poprzedzony nieudanym wciśnięciem spacji

### Musimy wybrać sygnał z której elektrody będziemy analizować. Zobaczmy jak wygląda nasz surowy sygnał 

In [ ]:
t = np.linspace(0,len(df)/250,len(df))

plt.plot(t,df['Cz'])
plt.xlabel('Czas [s]')
plt.ylabel('[uV]')

### Aby coś zobaczyć, trzeba usunąć zakłócenia z sygnału przy pomocy filtracji. 

In [ ]:
przef = ag.gornoprzepustowy(df['Cz'],250,0.1)
przef2 = ag.pasmowozaporowy(przef,250,48,52)
przef3 = ag.pasmowoprzepustowy(przef2,250,1,40)

In [ ]:
przef3

In [ ]:
plt.plot(t,przef3)
plt.xlabel('Czas [s]')
plt.ylabel('[uV]')

<span style="color:red">Proszę zwrócić uwagę na to, że przy szukaniu potencjałów pracujemy na sygnale w dziedzinie czasu (szukamy pików, które wystąpiły *po określonym czasie* od wystąpienia bodźców), dlatego transformacja Fouriera nie będzie nam dzisiaj potrzebna. </span>

### Następnie musimy znaleźć miejsca w czasie (indeksy) kiedy wystąpił bodziec. 

W kolumnie event pojawiają się flagi, kiedy występowało interesujące nas zdarzenie. Nas interesuje nr 4.

In [ ]:
# wyszukujemy indeksy w których pojawia się flaga '4', a wcześniejszy indeks ma flagę '0' 
poczatki_zdarzen = []
for i in range(len(df)):
    if df['event'][i]==4 and df['event'][i-1]==0:
        poczatki_zdarzen.append(i)

In [ ]:
#sprawdzamy liczbę takich zdarzeń
len(poczatki_zdarzen)

In [ ]:
poczatki_zdarzen

### Stwórzmy 1-sekundowe wykresy dla naszych zdarzeń

In [ ]:
indeks = poczatki_zdarzen[0]
fragment_sygnalu = przef3[indeks:indeks+250]

t = np.linspace(0,1,250)
plt.plot(t,fragment_sygnalu)

### Czasem dla pojedynczych przypadków załamki mogą być niewidoczne. Dlatego uśrednia się sygnał z kilku zdarzeń, żeby wzmocnić szukany efekt. 

In [ ]:
sredni_sygnal = np.zeros(250)

for i in range(10):
    sredni_sygnal += przef3[poczatki_zdarzen[i]:poczatki_zdarzen[i]+250]
sredni_sygnal/=10

In [ ]:
t = np.linspace(0,1,250)
plt.plot(t,sredni_sygnal)

### Dla porównania losowe fragmenty z sygnału

In [ ]:
losowe_indeksy = [68688, 50571, 58788, 95741, 19578, 15622, 95932, 9387, 30013, 93260]

# losowanie 10 indeksów
# losowe_indeksy = [random.randint(0,len(df)-250) for i in range(10)]

suma = np.zeros(250)
for indeks in losowe_indeksy:
    suma += przef3[indeks:indeks+250]
suma/=10

t = np.linspace(0,1,250)
plt.plot(t,suma)

### W jaki sposób automatycznie możemy ocenić czy wystąpił N400? 

###   Zadanie <img src="../images/pencil.png" width="20" align="left">

Pobierz plik 'sygnal_zadanie.csv'. Oceń które z podanych niżej indeksów zawierają sygnał N400. 


In [ ]:
indeksy_1 = [938, 13371, 22252, 31128, 54798, 92047, 98816, 102843, 103273, 123659]
indeksy_2 = [7262, 19641, 27905, 39668, 41581, 72379, 76208, 86419, 112203, 120647]
indeksy_3 = [6423, 28964, 39804, 61360, 77226, 92327, 101265, 114662, 116682, 123936]